In [2]:
!pip install koreanize-matplotlib

import koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 99.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train_path = '/content/drive/MyDrive/DF/train.csv'
test_path = '/content/drive/MyDrive/DF/test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452 entries, 0 to 2451
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2452 non-null   object 
 1   매물확인방식  2452 non-null   object 
 2   보증금     2452 non-null   float64
 3   월세      2452 non-null   int64  
 4   전용면적    1665 non-null   float64
 5   해당층     2223 non-null   float64
 6   총층      2436 non-null   float64
 7   방향      2452 non-null   object 
 8   방수      2436 non-null   float64
 9   욕실수     2434 non-null   float64
 10  주차가능여부  2452 non-null   object 
 11  총주차대수   1756 non-null   float64
 12  관리비     2452 non-null   int64  
 13  중개사무소   2452 non-null   object 
 14  제공플랫폼   2452 non-null   object 
 15  게재일     2452 non-null   object 
 16  허위매물여부  2452 non-null   int64  
dtypes: float64(7), int64(3), object(7)
memory usage: 325.8+ KB


In [9]:
train.head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일,허위매물여부
0,TRAIN_0000,현장확인,402500000.0,470000,NaN,NaN,15.0,서향,1.0,1.0,가능,40.0,96,t93Nt6I2I0,B플랫폼,2024-10-09,0
1,TRAIN_0001,현장확인,170500000.0,200000,NaN,3.0,4.0,남동향,2.0,1.0,불가능,NaN,0,q39iV5J4E6,D플랫폼,2024-12-26,0
2,TRAIN_0002,전화확인,114000000.0,380000,NaN,2.0,3.0,동향,1.0,1.0,불가능,NaN,0,b03oE4G3F6,A플랫폼,2024-11-28,0
3,TRAIN_0003,현장확인,163500000.0,30000,36.3,3.0,9.0,남동향,2.0,1.0,가능,13.0,10,G52Iz8V2B9,A플랫폼,2024-11-26,0
4,TRAIN_0004,현장확인,346000000.0,530000,NaN,3.0,3.0,동향,2.0,1.0,불가능,NaN,0,N45gM0M7R0,B플랫폼,2024-06-25,1


In [10]:
test.head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일
0,TEST_000,서류확인,223000000.0,530000,NaN,5.0,5.0,남향,1.0,1.0,불가능,NaN,7,Z86Th6S3K5,D플랫폼,2024-10-21
1,TEST_001,서류확인,150500000.0,590000,30.26,7.0,11.0,서향,1.0,1.0,불가능,16.0,11,G52Iz8V2B9,D플랫폼,2023-09-02
2,TEST_002,현장확인,47000000.0,200000,41.50,2.0,3.0,남서향,2.0,1.0,불가능,NaN,0,N45gM0M7R0,B플랫폼,2024-04-03
3,TEST_003,서류확인,133000000.0,250000,31.35,5.0,6.0,남향,2.0,1.0,가능,NaN,5,C41wx1K6U9,B플랫폼,2024-09-19
4,TEST_004,현장확인,108000000.0,380000,23.14,2.0,3.0,남향,1.0,1.0,불가능,NaN,0,Z68ZJ6F6L4,A플랫폼,2024-11-10


열별 결측치 개수

In [11]:
# df_cleaned = train.dropna(subset=['총층', '방수', '욕실수'])

In [18]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.9 MB/s eta 0:00:00


In [19]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

전처리

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
def preprocess_data(df, is_train=True, label_encoders=None):
    df = df.copy()

    # 날짜 데이터 처리 ('게재일' 분리)
    if '게재일' in df.columns:
        df['게재일_년'] = pd.to_datetime(df['게재일']).dt.year
        df['게재일_월'] = pd.to_datetime(df['게재일']).dt.month
        df['게재일_일'] = pd.to_datetime(df['게재일']).dt.day
        df.drop(columns=['게재일'], inplace=True)

    # 열 제거
    cols_to_drop = ['ID', 'index', '제공플랫폼']
    for col in cols_to_drop:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)

    # 보증금, 월세에 따른 전용면적 결측값 채우기
    df['보증금_월세_가중합'] = df['보증금'] + df['월세'] * 100
    ratio_mean = df.loc[df['전용면적'].notnull(), '전용면적'].mean() / df.loc[df['전용면적'].notnull(), '보증금_월세_가중합'].mean()

    def fill_area(row):
        if np.isnan(row['전용면적']):
            return row['보증금_월세_가중합'] * ratio_mean
        return row['전용면적']

    df['전용면적'] = df.apply(fill_area, axis=1)
    df.drop(columns=['보증금_월세_가중합'], inplace=True)

    # 총층으로 총주차대수 결측값 채우기
    if '총층' in df.columns and '총주차대수' in df.columns:
        knn_imputer = KNNImputer(n_neighbors=3)
        df[['총층', '총주차대수']] = knn_imputer.fit_transform(df[['총층', '총주차대수']])

    # 범주형 데이터 인코딩
    label_cols = ['매물확인방식', '방향', '주차가능여부', '중개사무소']
    if is_train:
        label_encoders = {}
        for col in label_cols:
            if col in df.columns:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col].astype(str))
                label_encoders[col] = le
    else:
        for col in label_cols:
            if col in df.columns and col in label_encoders:
                le = label_encoders[col]
                # 새로운 값 처리: 보지 못한 값은 -1로 변환
                df[col] = df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)

    # 중간값으로 나머지 결측값 처리
    for col in df.columns:
        if df[col].dtype in ['float64', 'int64']:
            df[col].fillna(df[col].median(), inplace=True)

    if is_train:
        return df, label_encoders
    else:
        return df


In [24]:
from sklearn.impute import KNNImputer

# Train 데이터 전처리
train_processed, label_encoders = preprocess_data(train, is_train=True)

# Test 데이터 전처리
test_processed = preprocess_data(test, is_train=False, label_encoders=label_encoders)

# Train과 Test의 열 정렬
missing_cols = set(train_processed.columns) - set(test_processed.columns)
for col in missing_cols:
    test_processed[col] = 0

# 열 순서 맞추기
test_processed = test_processed[train_processed.columns.drop('허위매물여부')]

<ipython-input-23-7cfaf86ffa45>:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
<ipython-input-23-7cfaf86ffa45>:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [25]:
train_processed.head()

,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,허위매물여부,게재일_년,게재일_월,게재일_일
0,2,402500000.0,470000,61.594296,4.0,15.0,7,1.0,1.0,0,40.000000,96,240,0,2024,10,9
1,2,170500000.0,200000,26.103923,3.0,4.0,0,2.0,1.0,1,1.333333,0,228,0,2024,12,26
2,1,114000000.0,380000,20.828327,2.0,3.0,3,1.0,1.0,1,1.333333,0,150,0,2024,11,28
3,2,163500000.0,30000,36.300000,3.0,9.0,0,2.0,1.0,0,13.000000,10,31,0,2024,11,26
4,2,346000000.0,530000,54.674358,3.0,3.0,3,2.0,1.0,1,1.333333,0,72,1,2024,6,25


In [26]:
test_processed.head()

,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,게재일_년,게재일_월,게재일_일
0,0,223000000.0,530000,42.499611,5.0,5.0,2,1.0,1.0,1,15.666667,7,145,2024,10,21
1,0,150500000.0,590000,30.260000,7.0,11.0,7,1.0,1.0,1,16.000000,11,31,2023,9,2
2,2,47000000.0,200000,41.500000,2.0,3.0,1,2.0,1.0,1,19.000000,0,72,2024,4,3
3,0,133000000.0,250000,31.350000,5.0,6.0,2,2.0,1.0,0,12.666667,5,12,2024,9,19
4,2,108000000.0,380000,23.140000,2.0,3.0,2,1.0,1.0,1,19.000000,0,142,2024,11,10


## 모델 선택

In [27]:
# 최신 버전 설치
!pip install --upgrade scikit-learn catboost lightgbm xgboost
!pip install --upgrade lightgbm
!pip install scikit-learn==0.24.2 lightgbm==3.2.1 xgboost==1.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 83.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# xg boost 적합

In [29]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

X = train_processed.drop(columns=['허위매물여부'])
y = train_processed['허위매물여부']


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

model = XGBClassifier(eval_metric='logloss',n_estimators=500, learning_rate=0.05, random_state=156)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

macro_f1 = f1_score(y_val, y_pred, average='macro')
print("\nMacro F1 Score:", macro_f1)


Macro F1 Score: 0.8941671461449943


# catboost 적합

In [30]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

X = train_processed.drop(columns=['허위매물여부'])
y = train_processed['허위매물여부']


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# CatBoost 모델 정의
model = CatBoostClassifier(eval_metric='Logloss', iterations=1200, learning_rate=0.04, random_state=42, verbose=0)

# 모델 학습
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_val)

# F1 Score 계산
macro_f1 = f1_score(y_val, y_pred, average='macro')
print("\nMacro F1 Score:", macro_f1)



Macro F1 Score: 0.9059304703476483


In [31]:
# 클래스 불균형 처리
pos_weight = len(y[y == 0]) / len(y[y == 1])  # 부정/긍정 샘플 비율
model = CatBoostClassifier(
    eval_metric='Logloss',
    iterations=1200,
    learning_rate=0.04,
    random_state=42,
    verbose=0,
    scale_pos_weight=pos_weight
)
model.fit(X_train, y_train)
# 예측
y_pred = model.predict(X_val)

# F1 Score 계산
macro_f1 = f1_score(y_val, y_pred, average='macro')
print("\nMacro F1 Score:", macro_f1)



Macro F1 Score: 0.9199986621849864


In [ ]:
# 테스트 데이터 예측
test_predictions = model.predict(test_processed)

# 결과 저장
output = pd.DataFrame({'ID': test.iloc[:, 0], '허위매물여부': test_predictions})
output.to_csv('result_submission.csv', index=False)
print("result_submission.csv 파일이 생성되었습니다.")

In [ ]:
test_predictions

In [ ]:
output